# Chapter 1

### Function

```
# Example 1 : Function without args or kwargs
def func_name(param1, param2=1):
    """Function Definition.""" # Function definition Docstring
    
    new_value1 = param1 ** param2
    new_value2 = param2 ** param1
    new_tuple = (new_value1, new_value2) # Binding results in a tuple
    
    return new_tuple # Returning multiple results

func_name(2,3) # (8,9)

# Example 2 : Function with *args (a tuple of parameters)
def add_all(*args):
    """Sum all values in *args together."""

    sum_all = 0
    for num in args: # iterate over args tuple
        sum_all += num

    return sum_all

add_all(5, 10, 15, 20) # 50

# Example 3 : Function with **kwargs (a dictionary of parameters)
def print_all(**kwargs):
    """Print out key-value pairs in **kwargs."""

    for key, value in kwargs.items(): # iterate over kwargs dictionary
        print(f"key = {key}, value = {value}")

print_all(name="dumbledore") # key = name, value = dumbledore

# Example 4 : Function as return value
def get_function():
    def print_me(s):
        print(s)

    return print_me

new_func = get_function() # print_me is now referenced by new_func
# calling new_func will now call print_me
new_func('This is a sentence.')
```

### Function docstring

```
# DRY : make functions that do only one thing at a time
# None is a good choice for default argument
# See documentation
print(func_name.__doc__)
# Alternative way
import inspect
print(inspect.getdoc(func_name))
```

# Chapter 2

### Function based Context Manager

```
@contextlib.contextmanager
def database (url):
    # set up database connection
    db = postgres.connect(url)

    try:
        yield db
    except:
        print("Something is Wrong")
    finally:
        # tear down database connection
        db.disconnect()
url = 'http://datacamp.com/data'

with database(url) as my_db:
    course_list = my_db.execute( 'SELECT * FROM courses')
```

### Class based context manager

```
    
#### Class based context manager ##########
from contextlib import contextmanager

class DatabaseManager:
    def __init__(self, url):
        self.url = url
        self.db = None

    def connect(self):
        # set up database connection
        self.db = postgres.connect(self.url)
        return self.db

    def disconnect(self):
        # tear down database connection
        if self.db:
            self.db.disconnect()

    def __enter__(self):
        # Called when entering the 'with' block
        return self.connect()

    def __exit__(self, exc_type, exc_value, traceback):
        # Called when exiting the 'with' block
        self.disconnect()

        # Handle exceptions
        if exc_type is not None:
            print("Something is Wrong")

# Example usage:
url = 'http://datacamp.com/data'

with DatabaseManager(url) as my_db:
    try:
        course_list = my_db.execute('SELECT * FROM courses')
        # Perform operations with the database connection
    except Exception as e:
        # Handle specific exceptions if needed
        print(f"Exception: {e}")


```

### Nested context

- Best practice :  Connect the nested context using a function

```
def copy(src, dst):
    """Copy the contents of one file to another.
    Args:
    src (str): File name of the file to be copied.
    dst (str): Where to write the new file.
    """
# Open both files
with open(src) as f_src:
    with open(dst, 'w') as f_dst:
    # Read and write each line, one at a time
        for line in f_src:
            f_dst.write(line)
```

# Chapter 3

### Scope

```
# Change global variable value in local scope

global global_var
global_var = other_val ** 2

# Change outer function variable value in inner function of a nested function
# nonlocal variables are the variables that reside in parent function in a nested function
nonlocal outer_var
outer_var = 2

# Presedence : Local > Enclosing > Global > Built-in
```

### Functions as return values

```
def get_function():
    def print_me(s):
        print(s)

    return print_me

new_func = get_function() # print_me is now referenced by new_func
# calling new_func will now call print_me
new_func('This is a sentence.')
```

### Closures

```
### Closure: Attaching nonlocal variables to nested functions
### When there is a nested function where inner function is dependent on
### nonlocal variable of outer function, closure helps to contain the variable
### of that outer function inside a tuple to supply it later to
### the inner function when needed

def foo():
    a = 5
    def bar():
        print(a)
    return bar

func = foo()
func() # 5

# What is closure : A tuple that contains nonlocal variables
type(func.__closure__)
# See the number of nonlocal variables
len(func.__closure__)
# See values of closures
func.__closure__[0].cell_contents
```

### Decorator

```
### Decorators: Decorators are nothing but nested functions, where the parent function 
### takes only a function as input and the child function modifies the behavior of input function

from functools import wraps
def parent_func(input_func):
    @wraps(input_func)
    def child_func(*args, **kwargs):
        return input_func(*args, **kwargs)
    return child_func

@parent_func
def some_func(a, b):
    return a * b

some_func(1, 5) 


```

# Chapter 4

### Decorator Factory

```
### Decorator Factory: An outermost function that wraps around a decorator and takes argument for 
### the decorator since the decorator itself cannot take any argument other than a single function

from functools import wraps
def paramholder_func( p1, p2)
    def parent_func(input_func):
        @wraps(input_func)
        def child_func(*args, **kwargs):
            return input_func(*args, **kwargs)
        return child_func
    return parent_func

@parent_func(1,2)
def some_func(a, b):
    return a * b

```

### Example 1 : Timer

```
import time
from functools import wraps
def decorator(func):
    """A decorator that prints how long a function took to run."""
    # Use @wraps to bind function's meta data like docstring / function name etc
    @wraps(func)
    # Define the wrapper function to return.
    def changed_func(*args, **kwargs):
        t_start = time.time()
        # Call the decorated function and store the result.
        result = func(*args, **kwargs)
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
        return result
    return changed_func

@decorator
def sleep_n_seconds(n):
    time.sleep(n)

sleep_n_seconds(5) # 5.00001
```

### Example 2 : Decorator with parameter (Decorator factory)

```
from functools import wraps
# an outer function that takes argument
def decorator_factory(n):
    """Define and return a decorator"""
    def decorator(func):
        @wraps(func)
        def changed_func(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return changed_func
    return decorator

run_three_times = decorator_factory(3)
@run_three_times
def print_sum(a, b):
    print(a + b)

@decorator_factory(3)
def print_sum(a, b):
    print(a + b)
```